In [1]:
%matplotlib qt
import numpy as np 
import matplotlib.pyplot as plt
from spec_im import gui_fname, plot_pl_summary, plot_pl_3d_summary, plot_si_bands, plot_bss_results, plot_decomp_results
from spec_im import SpectralImage, PLSpectralImage, PL3DSpectralImage
import math

# pl_3d_plot
Plots a 3D spectral image. 

## 1. Load file

In [2]:
# fname = gui_fname()
fname = 'D:/Chris/BBOD_Share/uv_microscope/190430 confocal GaN pyramid uvpl/190424_163834_oo_asi_hyperspec_3d_scan.h5'

In [3]:
si = PL3DSpectralImage(fname=fname)
#si.plot_spec()

Load from D:/Chris/BBOD_Share/uv_microscope/190430 confocal GaN pyramid uvpl/190424_163834_oo_asi_hyperspec_3d_scan.h5 complete.
5 x 41 x 41 spatial x 1044 spectral points


In [ ]:
plt.figure()
si = si[200:950]
si.plot_spec()

In [4]:
plt.figure()
si.set_background(lims=(700, 900))
si.plot_spec()

-8.100746268656621


In [5]:
si = si[356:600]
plt.figure()
si.plot_spec()

## 2. Visualize file

In [ ]:
plot_pl_3d_summary(si, num_rows=3);

In [6]:
s, fsi = si.to_signal()

In [7]:
s.plot()

In [ ]:
plt.figure()
fsi.plot(cbar_orientation='vertical', cbar_position='right')

## 3. Hyperspy decomposition

In [8]:
# perform principal component analysis, look at the explained variance
s.decomposition(algorithm='svd')
s.plot_explained_variance_ratio()

In [ ]:
s.plot_decomposition_results()

In [17]:
COMPS = 4

In [18]:
# take a look at orthogonalized 
plot_bss_results(s,fsi,title='BSS',cmap='viridis',fig_rows=min((5, COMPS)),components=COMPS,max_iter=2000)

[<Figure size 1000x1000 with 12 Axes>]

In [19]:
sc = s.get_decomposition_model(components=4)
sc.plot()

In [20]:
(s-sc).plot()

In [ ]:
list(sc.__dict__.keys())

In [21]:
sc_si = fsi.copy(signal=sc)
sc_si.set_background()
nzsc = sc_si.to_signal()

7.140088634516015


In [23]:
plot_decomp_results(nzsc, fsi, title='NMF', cmap='viridis', fig_rows=4, algorithm='nmf', output_dimension=8)

[<Figure size 1000x1000 with 12 Axes>, <Figure size 1000x1000 with 12 Axes>]

## A1. sklearn based native decomposition

In [ ]:
si.decomposition(algorithm='svd', output_dimension=0.9999)

In [ ]:
si.plot_explained_variance_ratio()

In [ ]:
si.blind_source_separation(number_of_components=10, max_iter=2000)

In [ ]:
ii = 0
slice_list = si.get_slice_list()
loadings = si.get_bss_loadings()
factors = si.get_bss_factors()
num_factors = factors.shape[0]

num_rows = 5
num_figures = math.ceil(float(num_factors)/num_rows)
print(loadings.shape, factors.shape, si.spec_x_array.shape)
(nz, ny, nx, nf) = np.shape(si.spec_im)
num_cols = 3 + nz
show_scalebar=False

for jj in range(num_figures):
    plt.figure()
    for kk in range(num_rows):
        if jj*num_rows + kk >= num_factors:  
            break
        index = kk + jj*num_rows
        # print(num_rows, num_cols, 3*kk + 1)
        for ii in range(nz):
            plt.subplot(num_rows, num_cols, num_cols*kk + 1 + ii)
            if kk == 0:
                z_pos = 'z = %0.2f $\mu$m' % ((si.z_array[ii]-si.z_array[0])*1e3)
            else:
                z_pos = ''
                
            if kk==0 and ii==0:
                show_scalebar=True
            else:
                show_scalebar=False
            
            si._plot(loadings[index, ii, :, :], cbar_orientation='vertical', cbar_position='right',
                     title='%s' % z_pos, show_scalebar=show_scalebar)
            # print(num_rows, num_cols, (3*kk + 2, 3*kk+3))
        plt.subplot(num_rows, num_cols, (num_cols*kk + nz + 1, num_cols*kk + nz + 2))
        si._plot_spec(factors[index,:])
        plt.title('%d' % index)
        

In [ ]:
slice_list = si.get_slice_list()
plt.figure()
si.get_slice(slice_list[1])[330:360].plot()

In [ ]:
mask = si.get_slice(slice_list[2])[330:360].spec_im.sum(axis=-1) < 7.95e4
print(mask.shape)
plt.figure()
plt.imshow(mask)
si.apply_mask(mask)

In [ ]:
# mask2 = np.empty(np.shape(si.spec_im.sum(axis=-1)))
# mask2[kk in range(np.size(si.z_array)), :, :, :] = mask
# #si.apply_mask(mask2)
# print('mask size', np.size(mask2), 'nonzero values in mask', np.count_nonzero(mask2.flatten()), '', np.size(si.spec_im.sum(axis=-1)))

In [ ]:
si.decomposition(algorithm='svd', output_dimension=0.9999)
si.plot_explained_variance_ratio()

In [ ]:
si.blind_source_separation(number_of_components=10)

In [ ]:
ii = 0
slice_list = si.get_slice_list()
loadings = si.get_bss_loadings()
factors = si.get_bss_factors()
num_factors = factors.shape[0]

num_rows = 5
num_figures = math.ceil(float(num_factors)/num_rows)
print(loadings.shape, factors.shape, si.spec_x_array.shape)
(nz, ny, nx, nf) = np.shape(si.spec_im)
num_cols = 3 + nz
show_scalebar=False

for jj in range(num_figures):
    plt.figure()
    for kk in range(num_rows):
        if jj*num_rows + kk >= num_factors:  
            break
        index = kk + jj*num_rows
        # print(num_rows, num_cols, 3*kk + 1)
        for ii in range(nz):
            plt.subplot(num_rows, num_cols, num_cols*kk + 1 + ii)
            if kk == 0:
                z_pos = 'z = %0.2f $\mu$m' % ((si.z_array[ii]-si.z_array[0])*1e3)
            else:
                z_pos = ''
                
            if kk==0 and ii==0:
                show_scalebar=True
            else:
                show_scalebar=False
            
            si._plot(loadings[index, ii, :, :], cbar_orientation='vertical', cbar_position='right',
                     title='%s' % z_pos, show_scalebar=show_scalebar)
            # print(num_rows, num_cols, (3*kk + 2, 3*kk+3))
        plt.subplot(num_rows, num_cols, (num_cols*kk + nz + 1, num_cols*kk + nz + 2))
        si._plot_spec(factors[index,:])
        plt.title('%d' % index)
        

In [ ]:
sc = s.get_decomposition_model(components=10)
sc.plot()

In [ ]:
sc = s.get_decomposition_model(components=4)
sc.plot()